In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/similar_logic/instr_reg/v1/without_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/similar_logic/instr_reg/v1/without_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'ctrl_flag', 'type': 'input port'}, {'id': 3, 'content': 'bus_data', 'type': 'input port'}, {'id': 4, 'content': 'reg1_out', 'type': 'output port'}, {'id': 5, 'content': 'reg2_out', 'type': 'output port'}, {'id': 6, 'content': 'reg1', 'type': 'submodule'}, {'id': 7, 'content': 'reg2', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[3, 2, 0, 1, 3, 2, 0, 1, 6, 7], [6, 6, 6, 6, 7, 7, 7, 7, 4, 5]]\n, A data bus control module that processes and directs input signals across multiple registers based on control flags. The module captures data from a bus, storing it into appropriate registers, and extracts specific fields from the registers to be sent to output interfaces.\n\nModule name:\n    data_bus_control\n\nInput ports:\n    clk: Clock signal for synchronization.\n    rst: Active low reset s

In [6]:
conv[0]['conversations'][1]['value']

"module data_bus_control (\n    input clk,\n    input rst,\n    input [1:0] ctrl_flag,\n    input [15:0] bus_data,\n    output [7:0] reg1_out,\n    output [7:0] reg2_out\n);\n\n    // Submodule for register storage\n    wire [7:0] reg1_data, reg2_data;\n\n    register_storage reg1 (\n        .clk(clk),\n        .rst(rst),\n        .ctrl_flag(ctrl_flag),\n        .data_in(bus_data[7:0]), // Lower 8 bits of bus data\n        .data_out(reg1_data),\n        .ctrl_code(2'b01) // Control for first register\n    );\n\n    register_storage reg2 (\n        .clk(clk),\n        .rst(rst),\n        .ctrl_flag(ctrl_flag),\n        .data_in(bus_data[15:8]), // Upper 8 bits of bus data\n        .data_out(reg2_data),\n        .ctrl_code(2'b10) // Control for second register\n    );\n\n    // Output assignments\n    assign reg1_out = reg1_data;\n    assign reg2_out = reg2_data;\n\nendmodule\n\n// Submodule for register storage\nmodule register_storage (\n    input clk,\n    input rst,\n    input [1:0] 

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'clk', 'type': 'input port'}, {'id': 1, 'content': 'rst', 'type': 'input port'}, {'id': 2, 'content': 'ctrl_flag', 'type': 'input port'}, {'id': 3, 'content': 'bus_data', 'type': 'input port'}, {'id': 4, 'content': 'reg1_out', 'type': 'output port'}, {'id': 5, 'content': 'reg2_out', 'type': 'output port'}, {'id': 6, 'content': 'reg1', 'type': 'submodule'}, {'id': 7, 'content': 'reg2', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[3, 2, 0, 1, 3, 2, 0, 1, 6, 7], [6, 6, 6, 6, 7, 7, 7, 7, 4, 5]]\n, A data bus control module that processes and directs input signals across multiple registers based on control flags. The module captures data from a bus, storing it into appropriate registers, and extracts specific fields from the registers to be sent to output interfaces.\n\nModule name:\n    data_bus_control\n\nInput ports:\n    clk: Clock signal for synchronization.\n    rst: Active low reset s

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [9]:
# conv[100]['conversations'][0]['value']

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/similar_logic/instr_reg/v1/with_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [12]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [13]:
len(new_conv)

480

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148